In [1]:
!pip install -q tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 61.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_text as text

In [3]:
# !gdown 1Jn0TYbliRDCd7U-q2ycOyMLWsYdaAm_E

In [4]:
!gdown 1RrqfGjIvryoyDTuflwia07AYEKiOYiZd

Downloading...
From: https://drive.google.com/uc?id=1RrqfGjIvryoyDTuflwia07AYEKiOYiZd
To: /content/cnn_lstm_hate_speech_bert.zip
100% 111M/111M [00:04<00:00, 26.9MB/s]


In [5]:
# !gdown --id 1xtgRYo4BsdHDK0eH6s_Nukt_LyYx4QEL

In [6]:
!gdown --id 1OmqM7ZmehdxZYZYHEkUDD06AsBy3n2u1

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1OmqM7ZmehdxZYZYHEkUDD06AsBy3n2u1
To: /content/result.csv
100% 14.0/14.0 [00:00<00:00, 67.4kB/s]


In [7]:
!unzip cnn_lstm_hate_speech_bert.zip -d /content

Archive:  cnn_lstm_hate_speech_bert.zip
   creating: /content/cnn_lstm_hate_speech_bert/
 extracting: /content/cnn_lstm_hate_speech_bert/fingerprint.pb  
   creating: /content/cnn_lstm_hate_speech_bert/variables/
  inflating: /content/cnn_lstm_hate_speech_bert/variables/variables.data-00000-of-00001  
  inflating: /content/cnn_lstm_hate_speech_bert/variables/variables.index  
  inflating: /content/cnn_lstm_hate_speech_bert/saved_model.pb  
   creating: /content/cnn_lstm_hate_speech_bert/assets/
  inflating: /content/cnn_lstm_hate_speech_bert/assets/vocab.txt  
  inflating: /content/cnn_lstm_hate_speech_bert/keras_metadata.pb  


In [8]:
saved_model_path = 'cnn_lstm_hate_speech_bert'

In [9]:
reloaded_model = tf.saved_model.load(saved_model_path)

In [10]:
def process(sample_tweet):
    sample_tweet = tf.convert_to_tensor(sample_tweet)
    sample_tweet = sample_tweet.numpy()
    result = reloaded_model(sample_tweet)

    maximum = max(result[0])
    ans = np.where(result[0] == maximum)

    category = {0: 'hate speech', 1: 'offensive', 2: 'neutral'}

    return category[ans[0][0]]

In [11]:
sample_tweet = ['replace me with any text or comment you like']

In [12]:
if not sample_tweet[0]:
    print('please enter an input')
else:
    print(process(sample_tweet))

neutral


# **Streamlit**

In [13]:
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=d0ea0bd6f7ce3bf803c7a9f8bd27201368fd30bdf8a34dd972762d8f4b9b0255
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators


In [14]:
%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_text as text

saved_model_path = '/content/cnn_lstm_hate_speech_bert'

reloaded_model = tf.saved_model.load(saved_model_path)

category = {0: 'Hate speech', 1: 'Offensive', 2: 'Neutral'}

def process(sample_tweet):
    sample_tweet = tf.convert_to_tensor(sample_tweet)
    print(sample_tweet)
    sample_tweet = sample_tweet.numpy()
    result = reloaded_model(sample_tweet)
    print(result)

    maximum = max(result[0])
    ans = np.where(result[0] == maximum)

    return ans[0][0]


st.markdown("<h1 style='text-align: center; font-family: Sans-Serif;'>Tweet Detector</h1>", unsafe_allow_html=True)

with st.form("my_form"):
    st.markdown("<p style='font-family: Sans-Serif; font-size: 20px;'>Please enter a tweet:</p>", unsafe_allow_html=True)
    user_input = st.text_input('', label_visibility="collapsed")

    submitted = st.form_submit_button("Check")

    sample_tweet = [user_input]

    data = {'text': sample_tweet, 'type': []}

    if sample_tweet[0]:
        label = process(sample_tweet)

        if label == 0:
            st.markdown("<p style='font-family: Sans-Serif; font-size: 20px; color: red;'>Hate speech</p>", unsafe_allow_html=True)
        elif label == 1:
            st.markdown("<p style='font-family: Sans-Serif; font-size: 20px; color: orange;'>Offensive speech</p>", unsafe_allow_html=True)
        elif label == 2:
            st.markdown("<p style='font-family: Sans-Serif; font-size: 20px; color: green;'>Neutral</p>", unsafe_allow_html=True)

        data['type'] = [category[label]]
        new_df = pd.DataFrame(data)
        new_df.to_csv('/content/result.csv', mode='a', index=False, header=False)


Writing app.py


In [15]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.102s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [16]:
# !npm install -g npm

In [17]:
!streamlit run /content/app.py &>/content/logs.txt &

In [18]:
!curl ipv4.icanhazip.com

34.125.100.19


In [19]:
!wget -q -O - ipv4.icanhazip.com

34.125.100.19


In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 1.442s
your url is: https://short-rice-train.loca.lt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil

In [ ]:
shutil.copy('/content/result.csv', '/content/drive/MyDrive/model_result/results')

FileNotFoundError: ignored